In [ ]:
import csv

In [ ]:
cve_count_per_topic={'0.0':0,'1.0':0,'2.0':0,'3.0':0,'4.0':0,'5.0':0,'6.0':0,'7.0':0,'8.0':0,
             '9.0':0,'10.0':0,'11.0':0,'12.0':0,'13.0':0,'14.0':0,'15.0':0,'16.0':0,'17.0':0}

with open('labelling.csv', 'r', newline='',encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        topic = row['Dominant_Topic']
        cve_count_per_topic[topic] = cve_count_per_topic[topic]+1
        
for k,v in cve_count_per_topic.items():
    perc = v/122344
    print(k,v,perc)

cwe alignment

In [ ]:
tcwes={'1':'693','2':'20','4':'22','5':'326','6':'20','7':'200',
             '9':'200','10':'444','11':'119','12':'787','13':'79','14':'79','16':'787','17':'78'}
topic_cve_count={'1.0':0,'2.0':0,'4.0':0,'5.0':0,'6.0':0,'7.0':0,
             '9.0':0,'10.0':0,'11.0':0,'12.0':0,'13.0':0,'14.0':0,'16.0':0,'17.0':0}
alignment={'1.0':0,'2.0':0,'4.0':0,'5.0':0,'6.0':0,'7.0':0,
             '9.0':0,'10.0':0,'11.0':0,'12.0':0,'13.0':0,'14.0':0,'16.0':0,'17.0':0}
tp={}
fp={}
fn={}
nocwes={}
with open('labelling.csv', 'r', newline='',encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        topic = row['Dominant_Topic']
        pcwe = row['CWE'].split('/')
#         print(len(pcwe),pcwe)
        if len(pcwe)<=1: #if cwe does not exists
            if topic in nocwes:
                nocwes[topic]+=1
            else:
                nocwes[topic]=1
            continue
        if topic in tcwes:
            if len(pcwe)>1: #if cwe exists
                pcwe = pcwe[5].split('.')[0]
#                 print(pcwe)
                if tcwes[topic] == pcwe: #true positive match for pcwe and tcwe
#                     alignment[topic] = alignment[topic]+1
                    if pcwe in tp:
                        tp[pcwe]+=1
                    else:
                        tp[pcwe]=1
                else:#false negative for pcwe, false postive for tcwe
                    if pcwe in fn:
                        fn[pcwe]+=1
                    else:
                        fn[pcwe]=1
                    if tcwes[topic] in fp:
                        fp[tcwes[topic]]+=1
                    else:
                        fp[tcwes[topic]]=1


# for k,v in alignment.items():
#     total = topic_cve_count[k]
#     align = v/total
#     print(k,v,total, align)
for k,v in nocwes.items():
    print(k,v)

In [ ]:
print("True positives: ",len(tp))
for k,v in tp.items():
    print(k,v)
    
print("False positives: ",len(fp))
for k,v in fp.items():
    print(k,v)
    
    
print("False negatives: ",len(fn))
for k,v in fn.items():
    print(k,v)

obtain cvedetails.com information

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
with open('cvedetails_info.csv', 'a',  newline='') as file:
    writer = csv.writer(file)
    for y in range(2011,2022):
        yr = str(y)
        mainurl = 'https://www.cvedetails.com/vulnerability-list.php?vendor_id=0&product_id=0&version_id=0&page={page}&hasexp=0&opdos=0&opec=0&opov=0&opcsrf=0&opgpriv=0&opsqli=0&opxss=0&opdirt=0&opmemc=0&ophttprs=0&opbyp=0&opfileinc=0&opginf=0&cvssscoremin=0&cvssscoremax=0&year={year}&month=0&cweid=0&order=2&trc=4155&sha=0b1f4462b5c3ed3cf0192383b9ec116f581ed999'
        mainurl = mainurl.format(year=yr,page='1')
        r = requests.get(mainurl, allow_redirects=True)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        numpages = soup.find('div', id='pagingb').find_all('a')
        all_urls=[]
        for n in numpages:
            url = "https://www.cvedetails.com"
            url+=n['href']
            all_urls.append(url)
            
        print("YEAR: ", yr,len(all_urls))
        count = 1
        for a in all_urls:
            print(count)
            count+=1
            
            r = requests.get(a, allow_redirects=True)
            soup = BeautifulSoup(r.content, 'html.parser')
            rows = soup.find('table', id="vulnslisttable").find_all('tr',{"class": "srrowns"})
            for r in rows:
                tds = r.find_all('td')
                cveid= tds[1].text
                vultype = tds[4].text.strip()
                if vultype=='+Info':
                    vultype='Info'
                elif vultype=='+Priv':
                    vultype='Priv'
                cvss_score = tds[7].text
#                 print(cveid,vultype,len(vultype),cvss_score)
                writer.writerow([cveid,str(vultype),cvss_score])


cve details alignment

In [ ]:
cvetypes = {}
nocvetypes=[]
with open('cvedetails_info.csv', 'r', newline='') as cve:
    cvedetails = csv.DictReader(cve)
    for c in cvedetails:
        types = c['Types'].split(" ")
        if types[0]!="" and len(types)>0:
            temp=[]
            for t in types:
                if t!='Code' and t!='Trav.'and t!="Corr." and t!="R.Spl." and t!="#NAME?":
                    if t=='+Priv':
                        temp.append('Priv')
                    elif t=="+Info":
                        temp.append('Info')
                    else:
                        temp.append(t)
            
            cvetypes[c['CVE-ID']] = temp 
            print(temp)
        else:
            nocvetypes.append(c['CVE-ID'])
print(len(cvetypes))

In [ ]:
topics_types={'0':['Bypass','Info'],'1':['XSS', 'CSRF','Http'],'2':['DoS', 'Exec', 'Mem.'],'3':['Bypass','Info'],'4':['Dir.', 'Bypass'],'5':['Bypass', 'Info'],'6':['Bypass', 'Info'],'7':['Bypass', 'Info'],'8':['Bypass', 'Mem.', 'Exec'],
             '9':['Bypass', 'Info'],'10':['Bypass', 'Info', 'Http'],'11':['Overflow'],'12':['Overflow', 'Exec'],'13':['XSS', 'Exec'],'14':['XSS', 'CSRF'],'15':['Exec'],'16':['Priv', 'Overflow', 'Exec'],'17':['Sql']}
types_alignment={'0.0':0,'1.0':0,'2.0':0,'3.0':0,'4.0':0,'5.0':0,'6.0':0,'7.0':0,'8.0':0,
             '9.0':0,'10.0':0,'11.0':0,'12.0':0,'13.0':0,'14.0':0,'15.0':0,'16.0':0,'17.0':0}
topic_type_count={'0.0':0,'1.0':0,'2.0':0,'3.0':0,'4.0':0,'5.0':0,'6.0':0,'7.0':0,'8.0':0,
             '9.0':0,'10.0':0,'11.0':0,'12.0':0,'13.0':0,'14.0':0,'15.0':0,'16.0':0,'17.0':0}
actual_types = {'0.0':[],'1.0':[],'2.0':[],'3.0':[],'4.0':[],'5.0':[],'6.0':[],'7.0':[],'8.0':[],
             '9.0':[],'10.0':[],'11.0':[],'12.0':[],'13.0':[],'14.0':[],'15.0':[],'16.0':[],'17.0':[]}
tp={}
fp={}
fn={}
with open('labelling.csv', 'r', newline='',encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for t in reader:
        dom_top = t['Dominant_Topic']
        if t['CVE-id'] in cvetypes: #if cve in labelling.csv has cve types (NOT EMPTY)
#             topic_type_count[dom_top]+=1
            cvedetails_types = cvetypes[t['CVE-id']] #actual types
            labelled_types = topics_types[dom_top] #labelled types
            
            #compare every labelled type with every actual type, increae tp, fp, fn count as you go
            for l in labelled_types:
                for c in cvedetails_types: 
                    if l==c: #true positive, increase tp count for l
                        if l in tp:
                            tp[l]+=1
                        else: 
                            tp[l]=1
                    else: #false postive for labelled_types, false negative for cvedetails_types
                        if l in fp:
                            fp[l]+=1
                        else:
                            fp[l]=1
                        if c in fn:
                            fn[c]+=1
                        else:
                            fn[c]=1

#             act = actual_types[dom_top]
#             for c in cvedetails_types:
#                 if c not in act:
#                     actual_types[dom_top].append(c)
#             print(t['CVE-id'], count)

# for k,v in types_alignment.items():
#     total=topic_type_count[k]
#     act_types = actual_types[k]
#     print(k,v,total,v/total,act_types)
print("True positives: ",len(tp))
for k,v in tp.items():
    print(k,v)
    
print("False positives: ",len(fp))
for k,v in fp.items():
    print(k,v)
    
    
print("False negatives: ",len(fn))
for k,v in fn.items():
    print(k,v)

In [ ]:
topics_no_types={}
count=0
with open('labelling.csv', 'r', newline='',encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        print(count)
        count+=1
        if row['CVE-id'] in nocvetypes:
            dom = row['Dominant_Topic']
            if dom in topics_no_types:
                topics_no_types[dom]+=1
            else:
                topics_no_types[dom]=1

for k,v in topics_no_types.items():
    print(k,v)

In [ ]:
type_freq={}
with open('labelling.csv', 'r', newline='',encoding='utf-8') as top:
    topics = csv.DictReader(top)
    for t in topics:
        dom_top = t['Dominant_Topic']
        if dom_top == '6.0' and t['CVE-id'] in cvetypes:
            for c in cvetypes[t['CVE-id']]:
                if c in type_freq:
                    type_freq[c]+=1
                else:
                    type_freq[c]=1

for k,v in type_freq.items():
    print(k,v)

In [ ]:
#count number of cves without CWE by topic, and number of cves without cvedetails type by topic

no_cwe = {}
dom_tops = {}
with open('labelling.csv', 'r', newline='',encoding='utf-8') as file:
    topics = csv.DictReader(file)
    for t in topics:
        if ".html" not in t['CWE']:
            dom_top = t['Dominant_Topic']
            if dom_top in no_cwe:
                no_cwe[dom_top]+=1
            else:
                no_cwe[dom_top]=1
        dom_tops[t['CVE-id']] = t['Dominant_Topic']

for k,v in no_cwe.items():
    print(k,v)        

In [ ]:
no_type={}
with open('cvedetails_info.csv', 'r', newline='',encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for r in reader:
        if r['Types'] =="":
            if r['CVE-ID'] in dom_tops:
                topic = dom_tops[r['CVE-ID']]
                if topic in no_type:
                    no_type[topic]+=1
                else:
                    no_type[topic]=1
            else:
                print(r['CVE-ID'])
print("--------------------")
for k,v in no_type.items():
    print(k,v) 